In [2]:
"""flagged to be rev 
1.)339 Delhi Potatoes
2.)340
3.)353
"""

'flagged to be rev \n1.)339 Delhi Potatoes\n2.)340\n3.)353\n'

In [3]:
#modified till 445
#flagged 422 just pudla
#

Automating driver for scraping and downloading recipes' images.

In [4]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import io
import requests
from PIL import Image
import urllib3
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.action_chains import ActionChains

In [7]:
#335 to 668 incl

#checking if the image can load with a good value of timeout
def fetch_image_content_from_url(url,timeout):
    try:
        http = urllib3.PoolManager(timeout=urllib3.Timeout(connect=None, read=None, total=timeout))
        response = http.request('GET', url)
        return response.data
    except Exception as e:
        print("Exception occurred inside fetch_image_content_from_url\n",e)
        

#download particular image and saving as 'filename'   
def download_image(download_path,url,filename):
    try:
        image_content = fetch_image_content_from_url(url,200)#fetching content of the image with a 200s timeout loading time
        #requests.get(url,timeout=10)
        image_bytes = io.BytesIO(image_content)#converting to bytes
        file_path = os.path.join(download_path,filename)#creating filepath for image
        image = Image.open(image_bytes)#opening the image
        _image=image.convert("RGB")#converting to RGB format since png format(which contains RGBA) gives error while downloading
        with open(file_path,"wb") as f:
            _image.save(f,"JPEG")
    except Exception as e:
        print("Exception is ", e)
        
        
def download_images(download_folder_path,urls,recipe,recipe_index,train_imgs_count):
    try:
        recipe_folder_name = str(recipe_index) + "_" + recipe
        train_folder_path = os.path.join(download_folder_path,"train",recipe_folder_name)
        test_folder_path = os.path.join(download_folder_path,"test",recipe_folder_name)

        if not os.path.exists(train_folder_path):
            os.makedirs(train_folder_path)
        if not os.path.exists(test_folder_path):
            os.makedirs(test_folder_path)
            
        img_number = 1
        for url in urls:
            if img_number <= train_imgs_count:
                download_image(train_folder_path,url,str(img_number)+"_"+recipe+".jpg")
            else:
                download_image(test_folder_path,url,str(img_number)+"_"+recipe+".jpg")
            img_number+=1
    except Exception as e:
        print("Exception is ", e)
        #log_urls(image_urls,recipe_index_csv,recipe,"dwd_logs.txt",no_of_img_to_train)
        
def log_urls(image_urls,recipe_index_csv,recipe,path_to_download_logs,train_imgs_count,ignore_msgs=False):
    try:
        with open(path_to_download_logs,"a") as f:
            f.write(str(recipe_index_csv)+"->"+ recipe +"\n")
            f.write("$$$$\n") # mark the start delimiter
            logged_urls = 0
            for url in image_urls:
                if logged_urls < train_imgs_count:
                    f.write("train:>"+url+"\n")
                else:
                    f.write("test:>" + url+"\n")
                logged_urls+=1
            f.write("$$$$\n")
            f.write("\n") # one empty line between end delimiter and new recipe
            f.flush()
        
    except Exception as e:
        if not ignore_msgs:
            print("Exception occurred while logging urls\n",e)
            
driver=None   
        
def create_driver_and_load(cService,Options):
    global driver
    try:
        driver = webdriver.Chrome(service=cService,options=Options)
        driver.maximize_window()
        driver.get('https://images.google.com/')
        time.sleep(7)
    except Exception as e :
        print("Exception occurred creating driver \n",e)         

# change to joinpath
PATH="C:\\Users\\Aarush Raj\\OneDrive\\Desktop\\img2rec\\venv\\chromedriver.exe"
chrome_service=webdriver.ChromeService(executable_path=PATH)
chrome_options=Options()
chrome_options.add_experimental_option("detach",True)


try:
    # wait for search results to load
    start = 440
    row_count= 5
    big_delay = 7
    small_delay = 3
    max_imgs = 10 
    extra_imgs = 5
    #batch_size = 2#change to 10 later
    no_of_img_to_train=8
    max_attempts_driver=0

    create_driver_and_load(chrome_service,chrome_options)
    df = pd.read_csv("C:\\Users\\Aarush Raj\\OneDrive\\Desktop\\img2rec\\DEEP-CHEF-PROJECT\\links_copy_main.csv",skiprows= start,nrows=row_count,names=['name','link'])
    d_log=open("dwd_logs.txt","a")
    
    # download images
    for i in range(len(df['name'])):
        recipe=df.loc[i,'name']
        recipe_name=recipe
        recipe=recipe_name + " Recipe OR " + recipe_name + " Dish"
        recipe_index_csv=i+start
        
        #d_log.write(str(recipe_index_csv)+"->"+ recipe +"\n")
        d_log.flush()
        
        search_box = driver.find_element(By.NAME,"q")
        search_box.clear()
        search_box.send_keys(recipe)
        search_box.send_keys(Keys.ENTER)
        time.sleep(10)
    
        image_urls=list()
        # click on the first image
        thumbnails = driver.find_elements(By.CLASS_NAME,"mNsIhb")
        
        
        if(len(thumbnails)<(max_imgs + extra_imgs)):
            print("quitting driver")
            driver.quit()
            print("creating new driver")
            create_driver_and_load(chrome_service,chrome_options)
            max_attempts_driver+=1
            if(max_attempts_driver>=3):
                raise
            else:
                continue
            
        
            
        x=len(image_urls)
        count=0
        for thumbnail in thumbnails:
            if(count==max_imgs):
                break
            
            try:
                action = ActionChains(driver)
                action.move_to_element(thumbnail).click().perform()
                # wait for image to load
                time.sleep(10)
            except StaleElementReferenceException:
                print("Stale element encountered while clicking thumbnail. Refreshing page...")
                """driver.refresh()
                thumbnails = driver.find_elements(By.CLASS_NAME,"mNsIhb")  # Refetch thumbnails
                thumbnail=thumbnails[i]"""
                #wait.until(ExpectedConditions.presenceOfElementLocated(By.CLASS_NAME("mNsIhb")));
                time.sleep(5)
                pass
            except: 
                continue  
            #print(thumbnail)
            pop_up_window = driver.find_elements(By.CLASS_NAME,"jlTjKd")  
            for pop_up_elements in pop_up_window:
                if pop_up_elements.tag_name == 'a':
                    # print("<a> tag obtained")

                    # get all the img tags under <a>
                    img_tags = pop_up_elements.find_elements(By.TAG_NAME,"img")

                    # print("img tags obtained")
                    for img in img_tags:
                        class_name=img.get_attribute("class")
                        if "iPVvYb" in class_name.strip():
                            image_url=img.get_attribute('src')
                            if(image_url not in image_urls):
                                if(requests.get(image_url).status_code==200):        
                                    image_urls.append(image_url)
                                    print("Found ",count)
                                    count+=1
                                else:
                                    print("not able to load img ")    
                                    continue
                                
                                # f.write("Found " + str(success_count) + "\n")
                            else:
                                print("Found Duplicate")
        
        download_images("C:\\Users\\Aarush Raj\\OneDrive\\Desktop\\img2rec\\DEEP-CHEF-PROJECT\\downloaded_images",image_urls,recipe_name,recipe_index_csv,no_of_img_to_train)
                    
        log_urls(image_urls,recipe_index_csv,recipe_name,"dwd_logs.txt",no_of_img_to_train)        
                # go back to search results
        time.sleep(7)        
    # close web browser
    #print(image_urls)
    d_log.close()
except Exception as e:
    print("Exception-",e)
    driver.quit()    

Found  0
Found  1
Found Duplicate
Found  2
Found  3
Found  4
Found  5
Found  6
not able to load img 
Found  7
Found  8
Found  9
Found  0
Found  1
Found Duplicate
Found  2
Found  3
Found  4
Found  5
Found  6
Found  7
Found Duplicate
Found Duplicate
Found  8
Found  9
Found  0
Found Duplicate
not able to load img 
Found  1
Found  2
Found  3
Found  4
Found  5
Found  6
Found  7
Found  8
Found  9
Found  0
Found  1
Found  2
not able to load img 
Found  3
Found  4
Found  5
Found  6
Found  7
not able to load img 
Found  8
Found  9
Found  0
Found  1
Found  2
Found  3
Found  4
not able to load img 
Found  5
Found  6
Found Duplicate
Found  7
Found Duplicate
Found  8
not able to load img 
Found  9
